# Important Libraries

In [1]:
# Initialization commands
%matplotlib inline

# Important libraries
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import seaborn as sns
import os

import numpy.random as npr # random sampling functions
import scipy.stats as sps # statistical functions
import matplotlib.pyplot as plt # Matlab plotting framework
import matplotlib.mlab as mlab
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.ticker as mticker
import time

# Important functions
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from mpl_toolkits.mplot3d import Axes3D
from numpy import array
from scipy.io import loadmat
from multiprocessing.dummy import Pool as ThreadPool
from scipy.cluster.vq import vq, kmeans, whiten
from matplotlib import colors as mcolors
from taylorDiagram import TaylorDiagram
colors = dict(mcolors.BASE_COLORS, **mcolors.CSS4_COLORS)

# Misc settings
sns.set(context="poster", style="whitegrid")

import warnings
warnings.filterwarnings('ignore')

basePath = os.getcwd()

import matplotlib.pylab as pylab
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (15, 5),
         'axes.labelsize': 50,
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

# Functions

## Model-specifc Functions:

### Covariance Matrix
Cholesky decomposition of a Hermitian positive-definite matrix is defined as $\Sigma = LDL^T$, where $L$ is any real-valued lower unit triangular matrix and $D$ is a diagonal matrix with positive elements. We allow the MVN proposal distribution to populate the elements of $L$ in such a way that all proposals provided by the jumping kernel are valid. The code below manipulates these raw MVN proposal locations to ensure the resulting $\Sigma$ is always a Hermitian positive-definite matrix.

$$\theta^* \sim\ MVN(\theta,\lambda I)$$

$$\Sigma = LDL^T = \left[ \begin{array}{cccc}
1 & 0 & 0 \\
\theta_{4} & 1 & 0 \\
\theta_{5} & \theta_{6} & 1 \\ \end{array} \right]
\left[ \begin{array}{cccc}
\exp(\theta_{1}) & 0 & 0 \\
0 & \exp(\theta_{2}) & 0 \\
0 & 0 & \exp(\theta_{3}) \\ \end{array} \right]
\left[ \begin{array}{cccc}
1 & \theta_{4} & \theta_{5} \\
0 & 1 & \theta_{6} \\
0 & 0 & 1 \\ \end{array} \right] =
\left[ \begin{array}{cccc}
\sigma_{1}^2 & \rho_{12}\sigma_{1}\sigma_{2} & \rho_{13}\sigma_{1}\sigma_{3} \\
\rho_{12}\sigma_{1}\sigma_{2} & \sigma_{2}^2 & \rho_{23}\sigma_{2}\sigma_{3} \\
\rho_{13}\sigma_{1}\sigma_{3} & \rho_{23}\sigma_{2}\sigma_{3} & \sigma_{3}^2 \\ \end{array} \right]$$

In [2]:
def tril2cov(x,m):
    
    D = np.diag(np.exp(x[:m]))
    L = np.zeros((m,m))
    L[np.tril_indices(m,-1)] = x[-(len(x)-m):]
    L[np.diag_indices(m)] = np.ones(m) 
    
    return np.dot(np.dot(L,D),L.T)

### Unpacking the Covariance Matrix After Sampling
While variance parameter estimates can be useful, we desire inferences for the common elements of the correpsonding covariance matrix in our multivariate Bayesian model. Since the sampler operates on raw Cholesky decomposition parameters and then reshapes them into a covariance matrix with a helper function (i.e., tril2cov), the posterior samples we obtain aren't easily interpreted. As a result, we must convert Cholesky decomposition parameters into the inuitive components of the covariance matrix.

$$\Sigma = \left[ \begin{array}{cccc}
\sigma_{1}^2 & \rho_{12}\sigma_{1}\sigma_{2} & \rho_{13}\sigma_{1}\sigma_{3} \\
\rho_{12}\sigma_{1}\sigma_{2} & \sigma_{2}^2 & \rho_{23}\sigma_{2}\sigma_{3} \\
\rho_{13}\sigma_{1}\sigma_{3} & \rho_{23}\sigma_{2}\sigma_{3} & \sigma_{3}^2 \\ \end{array} \right]
= \left[ \begin{array}{cccc}
1 & 0 & 0 \\
\theta_{4} & 1 & 0 \\
\theta_{5} & \theta_{6} & 1 \\ \end{array} \right]
\left[ \begin{array}{cccc}
\exp(\theta_{1}) & 0 & 0 \\
0 & \exp(\theta_{2}) & 0 \\
0 & 0 & \exp(\theta_{3}) \\ \end{array} \right]
\left[ \begin{array}{cccc}
1 & \theta_{4} & \theta_{5} \\
0 & 1 & \theta_{6} \\
0 & 0 & 1 \\ \end{array} \right] = LDL^T$$

### Correlation Matrix Elements: $\rho_{ij}=\frac{\Sigma_{ij}}{\sigma_{i}\sigma_{j}}$, where $i = j \implies \rho_{ii}= \rho_{jj}=1$
We note that the parameters corresponding to the diagnoals of this matrix should be identically one. Only the Pearson product-moment correlation coefficients should survive.

In [3]:
def cov2cor(x,d):
    
    x = x.as_matrix()
    n = x.shape[0] # number of samples
    p = d*(d+1)/2 # total number of unique elements in the covariance matrix
    m = d*(d-1)/2 # number of unique off-diagonal elements
    
    # preallocate arrays
    sig = np.zeros((n,d))
    pcc = np.zeros((n,m))
    
    for i in range(n):
        
        covM = tril2cov(x[i,-p:],d)
        sig[i] = np.sqrt(np.diag(covM))  
        covI = np.outer(sig[i],sig[i])
        corM = covM/covI
        pcc[i] = corM[np.tril_indices(d,-1)]
    
    return sig,pcc

## Helper Functions:

### Data Standardization: Z-Scores
The computational behavior of MLE and MCMC methods is improved when working with data has been shifted and scaled to have $\mu=0$ and $\sigma^2=1$ along each dimension of the data vector.

In [4]:
# center and/or scale data to have mean = 0 and variance = 1

def zscore(data, mode=0):
    
    mu = np.mean(data)
    sig = np.std(data)
    
    if mode == 0:
        
        z = (data - mu)/sig
        
    else:
        
        z = data/sig
    
    return z,mu,sig

In [5]:
# provide an inverse z-score transformation

def zreturn(data, mu, sig, dim=0):
    
    return sig[dim]*data + mu[dim]

### Credible Estimates: The Highest Density Interval/Region
Meaningful quantitative uncertainty statements are frequently delivered as interval estimates over the desired random variable(s). It is often convenient to idenitfy the narrowest such interval/region when summarizing the uncertainty associated with a statistical inference. The highest density interval is the narrowest interval that contains a specified probablistic mass fraction.   

In [6]:
# Code modified for Python from Doing Bayesian Data Analysis: A Tutorial with R and BUGS by John K. Kruschke
# Functions for approximating highest density intervals

def HDI(data, massFrac=0.95):
    
    data = np.reshape(data,(1,-1))
    sortData = np.sort(data).ravel()
    ciIdxInc = np.floor(massFrac*len(sortData))
    nCI = len(sortData) - ciIdxInc
    
    ciWidth = np.zeros((1,nCI)).ravel()
    
    for idx in range(int(nCI)):
        
        ciWidth[idx] = sortData[idx+ciIdxInc] - sortData[idx] 
    
    return sortData[np.argmin(ciWidth)],sortData[np.argmin(ciWidth)+ciIdxInc]

### KDE Belief Interpolation

In [7]:
def ykde(ax, xi):
    
    data_x, data_y = ax.lines[0].get_data()
    
    return np.interp(xi,data_x, data_y)

### Freedman–Diaconis Rule: Histogram Parameter Determination

In [8]:
# determine optimal histogram bin parameters using the FD rule
# mode 0 returns the optimal number of bins
# mode 1 returns the optimal bin width

def binFD(data, mode = 0):
    
    data = np.reshape(data,(1,-1))
    data = np.sort(data).ravel()
    bw = 2*sps.iqr(data)/(len(data)**(1/3))
    
    if mode == 0:
        
        funcAns = int(np.ptp(data)/bw)
        
    else:
                    
        funcAns = bw
        
    return funcAns

### Autocorrelation Function   

In [9]:
def ACF(data, lags):
    
    sac = np.zeros((lags))
    arg = data - np.mean(data)
    
    for i in range(lags):
        
        sac[i] = np.inner(np.reshape(arg[-(data.size-(i+1)):],(1,-1)),
                          np.reshape(arg[:(data.size-(i+1))],(1,-1)))
    
    return np.insert(sac/np.sum(arg**2),0,1)

### Empirical CDF Helper Function   

In [10]:
def ecdf(data):
    
    data = np.reshape(data,(1,-1))
    sortData = np.sort(data).ravel()

    return sortData, np.arange(1, len(sortData)+1)/float(len(sortData))

## Plotting Functions:

### Log-plot Tick Formatting

In [11]:
def log_tick_formatter(val, pos=None):
        
    return "{:.0e}".format(10**val)

### MCMC Diagnostics

In [12]:
def plot_diag(dx,burnVal,N,ix,basePath):
    
    plt.figure(figsize=(32,24))
    
    for j in range(nChains):
        
        pw = dx.Width[dx['Chain']==j]
        plt.semilogx(np.arange(1,N+1), pw, label = r'$Chain$ ' + str(j), lw = 3)
        
    plt.xlabel('Proposal',  fontsize = 40)
    plt.ylabel(r'Variance: log($\sigma^2_k$)',  fontsize = 40, color='black')
    plt.title('Adaptive Gaussian Kernel | Chain Length: ' + nStr + ' | ID: ' + mdl + ' ' + cityStr,  fontsize = 50)
    xLim = plt.xlim()
    yLim = plt.ylim()
    plt.plot([burnVal,burnVal], [yLim[0], yLim[1]], color='r', label=r'$Burn$' + '-' + r'$in$', lw = 3)
    plt.plot(xLim, [np.median(dx.Width),np.median(dx.Width)], label = '$\mu_{1/2}$', color='black', lw = 3, ls='--')
    plt.xlim(xmin=1,xmax=N)
    plt.legend(prop={'size':25});
    plt.savefig(basePath + '/' + mdl + '/' + ix + '/diag_pw.png');      
    plt.clf()
    
    plt.figure(figsize=(32,24))
    
    for j in range(nChains):
        
        ns = dx.Accept[dx['Chain']==j]
        plt.semilogx(np.arange(1,N+1), ns, label = r'$Chain$ ' + str(j), lw = 3)
        
    plt.xlabel('Proposal',  fontsize = 40)
    plt.ylabel('Acceptance Rate',  fontsize = 40, color='black')
    plt.title('Adaptive Gaussian Kernel | Chain Length: ' + nStr + ' | ID: ' + mdl + ' ' + cityStr,  fontsize = 50)
    xLim = plt.xlim()
    yLim = plt.ylim()
    plt.plot([burnVal,burnVal], [yLim[0], yLim[1]], color='r', label=r'$Burn$' + '-' + r'$in$', lw = 3)
    plt.plot(xLim, [np.mean(dx.Accept),np.mean(dx.Accept)], label = '$\mu_{1/2}$', color='black', lw = 3, ls='--')
    plt.xlim(xmin=1,xmax=N)
    plt.legend(prop={'size':25});
    plt.savefig(basePath + '/' + mdl + '/' + ix + '/diag_acc.png');      
    plt.clf()

### Correlation Coefficients

In [13]:
def pccAll(thin,mm,ix,basePath):
    
    sig,pcc = cov2cor(thin,mm)
    dfSig = pd.DataFrame(sig)
    dfPCC = pd.DataFrame(pcc)

    # sigma_1
    fig, ax1 = plt.subplots(figsize=(32, 24))
    dh = dfSig.iloc[:,0].tolist()
    nx, xx, px = ax1.hist(dh, bins = binFD(dh), normed=1, facecolor='grey', alpha=0.3)
    kde = sps.gaussian_kde(dh)
    yy = kde(xx)
    lns1 = ax1.plot(xx, yy, color = 'grey', label = r'$PDF$')
    yLim = ax1.get_ylim()
    lHDI,rHDI = HDI(dh)
    lIQR,rIQR = HDI(dh,0.5)
    hHDI = 0.5*(np.interp(lHDI,xx,yy) + np.interp(rHDI,xx,yy))
    hIQR = 0.5*(np.interp(lIQR,xx,yy) + np.interp(rIQR,xx,yy))
    cHDI = np.median(dh)
    xCDF,yCDF = ecdf(dh)
    yInt = np.interp(cHDI,xCDF, yCDF)
    lns2 = ax1.plot([lHDI,rHDI], [hHDI, hHDI], color='r', label=r'$HDI_{95\%}$', lw = 3, ls = '--')
    lns3 = ax1.plot([cHDI,cHDI], [yLim[0], np.interp(cHDI,xx,yy)], color='k', label=r'$\mu_{1/2}$', lw = 3, ls = '--')
    lns4 = ax1.plot([lIQR,rIQR], [hIQR, hIQR], color='b', label=r'$IQR$', lw = 3, ls = '--')
    ax1.set_xlabel(r'$\sigma_1$',  fontsize = 40)
    ax1.set_ylabel('Belief',  fontsize = 40)
    ax1.set_title(r'$\sigma_1$ Marginal Posterior | ' + mdl + ' ' + cStr + ' Samples (' + fStr + '% Thinning) | ID: ' + cityStr,  fontsize = 50)

    # second axis
    axa = ax1.twinx()
    axa.grid(b=None)
    lns5 = axa.plot(xCDF,yCDF, label = r'$CDF$', lw = 3, color = 'blue')
    yLimA = axa.get_ylim()
    axa.set_ylabel('Probability', fontsize = 40)
    axa.set_ylim(bottom=0)
    axa.tick_params('y', colors='blue')
    lns = lns1+lns5+lns2+lns4+lns3
    labs = [l.get_label() for l in lns]
    axa.legend(lns, labs, loc=2, prop={'size':25})

    # annotations
    ax1.annotate(r'$HDI_{95\%}$ = ' + str(np.around(rHDI-lHDI,decimals=3)), 
                 xy=((3*rHDI+lHDI)/4, hHDI), 
                 xytext=(rHDI, 2*yLim[1]/5),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    axa.annotate(r'$\mu_{1/2}$ = ' + str(np.around(cHDI,decimals=3)),
                 xy=(cHDI, yInt), 
                 xytext=(rHDI, yLimA[1]/2),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    ax1.annotate(r'$IQR$ = ' + str(np.around(rIQR-lIQR,decimals=3)), 
                 xy=(np.percentile(dh, 60), hIQR), 
                 xytext=(rHDI, 3*yLim[1]/5),
                 textcoords='data', arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    plt.savefig(basePath + '/' + mdl + '/' + ix + '/sigma_1.png');      
    plt.clf()
        
    # sigma_2
    fig, ax2 = plt.subplots(figsize=(32, 24))
    dh = dfSig.iloc[:,1].tolist()
    nx, xx, px = ax2.hist(dh, bins = binFD(dh), normed=1, facecolor='grey', alpha=0.3)
    kde = sps.gaussian_kde(dh)
    yy = kde(xx)
    lns1 = ax2.plot(xx, yy, color = 'grey', label = r'$PDF$')
    yLim = ax2.get_ylim()
    lHDI,rHDI = HDI(dh)
    lIQR,rIQR = HDI(dh,0.5)
    hHDI = 0.5*(np.interp(lHDI,xx,yy) + np.interp(rHDI,xx,yy))
    hIQR = 0.5*(np.interp(lIQR,xx,yy) + np.interp(rIQR,xx,yy))
    cHDI = np.median(dh)
    xCDF,yCDF = ecdf(dh)
    yInt = np.interp(cHDI,xCDF, yCDF)
    lns2 = ax2.plot([lHDI,rHDI], [hHDI, hHDI], color='r', label=r'$HDI_{95\%}$', lw = 3, ls = '--')
    lns3 = ax2.plot([cHDI,cHDI], [yLim[0], np.interp(cHDI,xx,yy)], color='k', label=r'$\mu_{1/2}$', lw = 3, ls = '--')
    lns4 = ax2.plot([lIQR,rIQR], [hIQR, hIQR], color='b', label=r'$IQR$', lw = 3, ls = '--')
    ax2.set_xlabel(r'$\sigma_2$',  fontsize = 40)
    ax2.set_ylabel('Belief',  fontsize = 40)
    ax2.set_title(r'$\sigma_2$ Marginal Posterior | ' + mdl + ' ' + cStr + ' Samples (' + fStr + '% Thinning) | ID: ' + cityStr,  fontsize = 50)

    # second axis
    axa = ax2.twinx()
    axa.grid(b=None)
    lns5 = axa.plot(xCDF,yCDF, label = r'$CDF$', lw = 3, color = 'blue')
    yLimA = axa.get_ylim()
    axa.set_ylabel('Probability', fontsize = 40)
    axa.set_ylim(bottom=0)
    axa.tick_params('y', colors='blue')
    lns = lns1+lns5+lns2+lns4+lns3
    labs = [l.get_label() for l in lns]
    axa.legend(lns, labs, loc=2, prop={'size':25})

    # annotations
    ax2.annotate(r'$HDI_{95\%}$ = ' + str(np.around(rHDI-lHDI,decimals=3)), 
                 xy=((3*rHDI+lHDI)/4, hHDI), 
                 xytext=(rHDI, 2*yLim[1]/5),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    axa.annotate(r'$\mu_{1/2}$ = ' + str(np.around(cHDI,decimals=3)),
                 xy=(cHDI, yInt), 
                 xytext=(rHDI, yLimA[1]/2),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    ax2.annotate(r'$IQR$ = ' + str(np.around(rIQR-lIQR,decimals=3)), 
                 xy=(np.percentile(dh, 60), hIQR), 
                 xytext=(rHDI, 3*yLim[1]/5),
                 textcoords='data', arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)

    plt.savefig(basePath + '/' + mdl + '/' + ix + '/sigma_2.png');      
    plt.clf()
    
    # sigma_3
    fig, ax3 = plt.subplots(figsize=(32, 24))
    dh = dfSig.iloc[:,2].tolist()
    nx, xx, px = ax3.hist(dh, bins = binFD(dh), normed=1, facecolor='grey', alpha=0.3)
    kde = sps.gaussian_kde(dh)
    yy = kde(xx)
    lns1 = ax3.plot(xx, yy, color = 'grey', label = r'$PDF$')
    yLim = ax3.get_ylim()
    lHDI,rHDI = HDI(dh)
    lIQR,rIQR = HDI(dh,0.5)
    hHDI = 0.5*(np.interp(lHDI,xx,yy) + np.interp(rHDI,xx,yy))
    hIQR = 0.5*(np.interp(lIQR,xx,yy) + np.interp(rIQR,xx,yy))
    cHDI = np.median(dh)
    xCDF,yCDF = ecdf(dh)
    yInt = np.interp(cHDI,xCDF, yCDF)
    lns2 = ax3.plot([lHDI,rHDI], [hHDI, hHDI], color='r', label=r'$HDI_{95\%}$', lw = 3, ls = '--')
    lns3 = ax3.plot([cHDI,cHDI], [yLim[0], np.interp(cHDI,xx,yy)], color='k', label=r'$\mu_{1/2}$', lw = 3, ls = '--')
    lns4 = ax3.plot([lIQR,rIQR], [hIQR, hIQR], color='b', label=r'$IQR$', lw = 3, ls = '--')
    ax3.set_xlabel(r'$\sigma_3$',  fontsize = 40)
    ax3.set_ylabel('Belief',  fontsize = 40)
    ax3.set_title(r'$\sigma_3$ Marginal Posterior | ' + mdl + ' ' + cStr + ' Samples (' + fStr + '% Thinning) | ID: ' + cityStr,  fontsize = 50)

    # second axis
    axa = ax3.twinx()
    axa.grid(b=None)
    lns5 = axa.plot(xCDF,yCDF, label = r'$CDF$', lw = 3, color = 'blue')
    yLimA = axa.get_ylim()
    axa.set_ylabel('Probability', fontsize = 40)
    axa.set_ylim(bottom=0)
    axa.tick_params('y', colors='blue')
    lns = lns1+lns5+lns2+lns4+lns3
    labs = [l.get_label() for l in lns]
    axa.legend(lns, labs, loc=2, prop={'size':25})

    # annotations
    ax3.annotate(r'$HDI_{95\%}$ = ' + str(np.around(rHDI-lHDI,decimals=3)), 
                 xy=((3*rHDI+lHDI)/4, hHDI), 
                 xytext=(rHDI, 2*yLim[1]/5),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    axa.annotate(r'$\mu_{1/2}$ = ' + str(np.around(cHDI,decimals=3)),
                 xy=(cHDI, yInt), 
                 xytext=(rHDI, yLimA[1]/2),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    ax3.annotate(r'$IQR$ = ' + str(np.around(rIQR-lIQR,decimals=3)), 
                 xy=(np.percentile(dh, 60), hIQR), 
                 xytext=(rHDI, 3*yLim[1]/5),
                 textcoords='data', arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)

    plt.savefig(basePath + '/' + mdl + '/' + ix + '/sigma_3.png');      
    plt.clf()
    
    # rho_12
    fig, ax1 = plt.subplots(figsize=(32, 24))
    dh = dfPCC.iloc[:,0].tolist()
    nx, xx, px = ax1.hist(dh, bins = binFD(dh), normed=1, facecolor='grey', alpha=0.3)
    kde = sps.gaussian_kde(dh)
    yy = kde(xx)
    lns1 = ax1.plot(xx, yy, color = 'grey', label = r'$PDF$')
    yLim = ax1.get_ylim()
    lHDI,rHDI = HDI(dh)
    lIQR,rIQR = HDI(dh,0.5)
    hHDI = 0.5*(np.interp(lHDI,xx,yy) + np.interp(rHDI,xx,yy))
    hIQR = 0.5*(np.interp(lIQR,xx,yy) + np.interp(rIQR,xx,yy))
    cHDI = np.median(dh)
    xCDF,yCDF = ecdf(dh)
    yInt = np.interp(cHDI,xCDF, yCDF)
    lns2 = ax1.plot([lHDI,rHDI], [hHDI, hHDI], color='r', label=r'$HDI_{95\%}$', lw = 3, ls = '--')
    lns3 = ax1.plot([cHDI,cHDI], [yLim[0], np.interp(cHDI,xx,yy)], color='k', label=r'$\mu_{1/2}$', lw = 3, ls = '--')
    lns4 = ax1.plot([lIQR,rIQR], [hIQR, hIQR], color='b', label=r'$IQR$', lw = 3, ls = '--')
    ax1.set_xlabel(r'$\rho_{12}$',  fontsize = 40)
    ax1.set_ylabel('Belief',  fontsize = 40)
    ax1.set_title(r'$\rho_{12}$ Marginal Posterior | ' + mdl + ' ' + cStr + ' Samples (' + fStr + '% Thinning) | ID: ' + cityStr,  fontsize = 50)

    # second axis
    axa = ax1.twinx()
    axa.grid(b=None)
    lns5 = axa.plot(xCDF,yCDF, label = r'$CDF$', lw = 3, color = 'blue')
    yLimA = axa.get_ylim()
    axa.set_ylabel('Probability', fontsize = 40)
    axa.set_ylim(bottom=0)
    axa.tick_params('y', colors='blue')
    lns = lns1+lns5+lns2+lns4+lns3
    labs = [l.get_label() for l in lns]
    axa.legend(lns, labs, loc=2, prop={'size':25})

    # annotations
    ax1.annotate(r'$HDI_{95\%}$ = ' + str(np.around(rHDI-lHDI,decimals=3)), 
                 xy=((3*rHDI+lHDI)/4, hHDI), 
                 xytext=(rHDI, 2*yLim[1]/5),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    axa.annotate(r'$\mu_{1/2}$ = ' + str(np.around(cHDI,decimals=3)),
                 xy=(cHDI, yInt), 
                 xytext=(rHDI, yLimA[1]/2),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    ax1.annotate(r'$IQR$ = ' + str(np.around(rIQR-lIQR,decimals=3)), 
                 xy=(np.percentile(dh, 60), hIQR), 
                 xytext=(rHDI, 3*yLim[1]/5),
                 textcoords='data', arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)

    plt.savefig(basePath + '/' + mdl + '/' + ix + '/rho_12.png');      
    plt.clf()
    
    # rho_13
    fig, ax2 = plt.subplots(figsize=(32, 24))
    dh = dfPCC.iloc[:,1].tolist()
    nx, xx, px = ax2.hist(dh, bins = binFD(dh), normed=1, facecolor='grey', alpha=0.3)
    kde = sps.gaussian_kde(dh)
    yy = kde(xx)
    lns1 = ax2.plot(xx, yy, color = 'grey', label = r'$PDF$')
    yLim = ax2.get_ylim()
    lHDI,rHDI = HDI(dh)
    lIQR,rIQR = HDI(dh,0.5)
    hHDI = 0.5*(np.interp(lHDI,xx,yy) + np.interp(rHDI,xx,yy))
    hIQR = 0.5*(np.interp(lIQR,xx,yy) + np.interp(rIQR,xx,yy))
    cHDI = np.median(dh)
    xCDF,yCDF = ecdf(dh)
    yInt = np.interp(cHDI,xCDF, yCDF)
    lns2 = ax2.plot([lHDI,rHDI], [hHDI, hHDI], color='r', label=r'$HDI_{95\%}$', lw = 3, ls = '--')
    lns3 = ax2.plot([cHDI,cHDI], [yLim[0], np.interp(cHDI,xx,yy)], color='k', label=r'$\mu_{1/2}$', lw = 3, ls = '--')
    lns4 = ax2.plot([lIQR,rIQR], [hIQR, hIQR], color='b', label=r'$IQR$', lw = 3, ls = '--')
    ax2.set_xlabel(r'$\rho_{13}$',  fontsize = 40)
    ax2.set_ylabel('Belief',  fontsize = 40)
    ax2.set_title(r'$\rho_{13}$ Marginal Posterior | ' + mdl + ' ' + cStr + ' Samples (' + fStr + '% Thinning) | ID: ' + cityStr,  fontsize = 50)

    # second axis
    axa = ax2.twinx()
    axa.grid(b=None)
    lns5 = axa.plot(xCDF,yCDF, label = r'$CDF$', lw = 3, color = 'blue')
    yLimA = axa.get_ylim()
    axa.set_ylabel('Probability', fontsize = 40)
    axa.set_ylim(bottom=0)
    axa.tick_params('y', colors='blue')
    lns = lns1+lns5+lns2+lns4+lns3
    labs = [l.get_label() for l in lns]
    axa.legend(lns, labs, loc=2, prop={'size':25})

    # annotations
    ax2.annotate(r'$HDI_{95\%}$ = ' + str(np.around(rHDI-lHDI,decimals=3)), 
                 xy=((3*rHDI+lHDI)/4, hHDI), 
                 xytext=(rHDI, 2*yLim[1]/5),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    axa.annotate(r'$\mu_{1/2}$ = ' + str(np.around(cHDI,decimals=3)),
                 xy=(cHDI, yInt), 
                 xytext=(rHDI, yLimA[1]/2),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    ax2.annotate(r'$IQR$ = ' + str(np.around(rIQR-lIQR,decimals=3)), 
                 xy=(np.percentile(dh, 60), hIQR), 
                 xytext=(rHDI, 3*yLim[1]/5),
                 textcoords='data', arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)

    plt.savefig(basePath + '/' + mdl + '/' + ix + '/rho_13.png');      
    plt.clf()
    
    # rho_23
    fig, ax3 = plt.subplots(figsize=(32, 24))
    dh = dfPCC.iloc[:,2].tolist()
    nx, xx, px = ax3.hist(dh, bins = binFD(dh), normed=1, facecolor='grey', alpha=0.3)
    kde = sps.gaussian_kde(dh)
    yy = kde(xx)
    lns1 = ax3.plot(xx, yy, color = 'grey', label = r'$PDF$')
    yLim = ax3.get_ylim()
    lHDI,rHDI = HDI(dh)
    lIQR,rIQR = HDI(dh,0.5)
    hHDI = 0.5*(np.interp(lHDI,xx,yy) + np.interp(rHDI,xx,yy))
    hIQR = 0.5*(np.interp(lIQR,xx,yy) + np.interp(rIQR,xx,yy))
    cHDI = np.median(dh)
    xCDF,yCDF = ecdf(dh)
    yInt = np.interp(cHDI,xCDF, yCDF)
    lns2 = ax3.plot([lHDI,rHDI], [hHDI, hHDI], color='r', label=r'$HDI_{95\%}$', lw = 3, ls = '--')
    lns3 = ax3.plot([cHDI,cHDI], [yLim[0], np.interp(cHDI,xx,yy)], color='k', label=r'$\mu_{1/2}$', lw = 3, ls = '--')
    lns4 = ax3.plot([lIQR,rIQR], [hIQR, hIQR], color='b', label=r'$IQR$', lw = 3, ls = '--')
    ax3.set_xlabel(r'$\rho_{23}$',  fontsize = 40)
    ax3.set_ylabel('Belief',  fontsize = 40)
    ax3.set_title(r'$\rho_{23}$ Marginal Posterior | ' + mdl + ' ' + cStr + ' Samples (' + fStr + '% Thinning) | ID: ' + cityStr,  fontsize = 50)

    # second axis
    axa = ax3.twinx()
    axa.grid(b=None)
    lns5 = axa.plot(xCDF,yCDF, label = r'$CDF$', lw = 3, color = 'blue')
    yLimA = axa.get_ylim()
    axa.set_ylabel('Probability', fontsize = 40)
    axa.set_ylim(bottom=0)
    axa.tick_params('y', colors='blue')
    lns = lns1+lns5+lns2+lns4+lns3
    labs = [l.get_label() for l in lns]
    axa.legend(lns, labs, loc=2, prop={'size':25})

    # annotations
    ax3.annotate(r'$HDI_{95\%}$ = ' + str(np.around(rHDI-lHDI,decimals=3)), 
                 xy=((3*rHDI+lHDI)/4, hHDI), 
                 xytext=(rHDI, 2*yLim[1]/5),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    axa.annotate(r'$\mu_{1/2}$ = ' + str(np.around(cHDI,decimals=3)),
                 xy=(cHDI, yInt), 
                 xytext=(rHDI, yLimA[1]/2),
                 textcoords='data',
                 arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    ax3.annotate(r'$IQR$ = ' + str(np.around(rIQR-lIQR,decimals=3)), 
                 xy=(np.percentile(dh, 60), hIQR), 
                 xytext=(rHDI, 3*yLim[1]/5),
                 textcoords='data', arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
    
    plt.savefig(basePath + '/' + mdl + '/' + ix + '/rho_23.png');      
    plt.clf()

### Posterior Parameter Belief

In [14]:
# Plotting function for parameter marginal posteriors

def plot_theta(th,dt,dth,burnVal,N,lags,idx,ix,basePath):
    
    md = []
    mn = []
    sd = []
    hd = []
    hc = []
    iq = []
    pm = []
    
    for i in range(th.shape[1]):
    
        # pdf and cdf distributions
        fig, ax1 = plt.subplots(figsize=(32, 24))
        dh = th.iloc[:,i]
        nx, xx, px = ax1.hist(dh, bins = binFD(dh), normed=1, facecolor='grey', alpha=0.3)
        kde = sps.gaussian_kde(dh)
        yy = kde(xx)
        lns1 = ax1.plot(xx, yy, color = 'grey', label = r'$PDF$')
        yLim = ax1.get_ylim()
        lHDI,rHDI = HDI(dh)
        lIQR,rIQR = HDI(dh,0.5)
        hHDI = 0.5*(np.interp(lHDI,xx,yy) + np.interp(rHDI,xx,yy))
        hIQR = 0.5*(np.interp(lIQR,xx,yy) + np.interp(rIQR,xx,yy))
        cHDI = np.median(dh)
        xCDF,yCDF = ecdf(dh)
        yInt = np.interp(cHDI,xCDF, yCDF)
        lns2 = ax1.plot([lHDI,rHDI], [hHDI, hHDI], color='r', label=r'$HDI_{95\%}$', lw = 3, ls = '--')
        lns3 = ax1.plot([cHDI,cHDI], [yLim[0], np.interp(cHDI,xx,yy)], color='k', label=r'$\mu_{1/2}$', lw = 3, ls = '--')
        lns4 = ax1.plot([lIQR,rIQR], [hIQR, hIQR], color='b', label=r'$IQR$', lw = 3, ls = '--')
        ax1.set_xlabel(r'$\theta_{' + repr(i+1) + '}$',  fontsize = 40)
        ax1.set_ylabel('Belief',  fontsize = 40)
        ax1.set_title(r'$\theta_{' + repr(i+1) + '}$ Marginal Posterior | ' + cStr + ' Samples (' + fStr + '% Thinning) | ID: ' + mdl + ' ' + cityStr,  fontsize = 50)
        
        # second axis
        axa = ax1.twinx()
        axa.grid(b=None)
        lns5 = axa.plot(xCDF,yCDF, label = r'$CDF$', lw = 3, color = 'blue')
        yLimA = axa.get_ylim()
        axa.set_ylabel('Probability', fontsize = 40)
        axa.set_ylim(bottom=0)
        axa.tick_params('y', colors='blue')
        lns = lns1+lns5+lns2+lns4+lns3
        labs = [l.get_label() for l in lns]
        axa.legend(lns, labs, loc=2, prop={'size':25})
        
        # annotations
        ax1.annotate(r'$HDI_{95\%}$ = ' + str(np.around(rHDI-lHDI,decimals=3)), 
                     xy=((3*rHDI+lHDI)/4, hHDI), 
                     xytext=(rHDI, 2*yLim[1]/5),
                     textcoords='data',
                     arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
        axa.annotate(r'$\mu_{1/2}$ = ' + str(np.around(cHDI,decimals=3)),
                     xy=(cHDI, yInt), 
                     xytext=(rHDI, yLimA[1]/2),
                     textcoords='data',
                     arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
        ax1.annotate(r'$IQR$ = ' + str(np.around(rIQR-lIQR,decimals=3)), 
                     xy=(np.percentile(dh, 60), hIQR), 
                     xytext=(rHDI, 3*yLim[1]/5),
                     textcoords='data', arrowprops=dict(facecolor='black', shrink=0.05), fontsize=40)
        
        plt.savefig(basePath + '/' + mdl + '/' + ix + '/post_' + str(i) + '.png');      
        plt.clf()
        
        fig, ax2 = plt.subplots(figsize=(32, 24))
        # trace plot
        for j in range(nChains):
            
            x = dt[dt['Chain']==j].iloc[:,3:]
            p1 = ax2.plot(np.arange(1,N+1), np.cumsum(x.iloc[:,i])/np.arange(1,N+1),label='Chain ' + str(j), lw = 3)
            ax2.scatter(iv, x.iloc[iv-1,i], marker = 'o', label = None, alpha = 0.3, color=p1[0].get_color())
            
        ax2.set_xscale('log')
        ax2.set_xlabel('Proposal',  fontsize = 40)
        ax2.set_ylabel('Parameter Location',  fontsize = 40)
        ax2.set_title(r'$\theta_{' + repr(i+1) + '}$ MCMC Trace Plot | Chain Length: ' + nStr + ' | ID: ' + mdl + ' ' + cityStr,  fontsize = 50)
        xLim = ax2.get_xlim()
        yLim = ax2.get_ylim()
        ax2.plot([burnVal,burnVal], [yLim[0], yLim[1]], color='r', label=r'$Burn$' + '-' + r'$in$', lw = 3)
        ax2.plot(xLim, [np.median(xh.iloc[:,i]),np.median(x.iloc[:,i])], color='grey', lw = 3, ls='--')
        ax2.legend(prop={'size':25});
        ax2.grid(True)
        ax2.set_ylim([yLim[0], yLim[1]])
        ax2.set_xlim(left=1,right=N)
        plt.savefig(basePath + '/' + mdl + '/' + ix + '/trace_' + str(i) + '.png');     
        plt.clf()
        
        fig, ax3 = plt.subplots(figsize=(32, 24))
        # parameter sample autocorrelation
        for j in range(nChains):
            
            x = dth.iloc[idx,:]
            x = x[x['Chain']==j].iloc[:,3:]
            ax3.plot(np.arange(lags+1),ACF(x.iloc[:,i],lags), label='Thinned Chain ' + str(j), lw = 3, ls = '--')
            
        ax3.plot(np.arange(lags+1),ACF(dh,lags), color = 'blue', label='Thinned Combined', lw = 3)
        ax3.plot(np.arange(lags+1),ACF(xh.iloc[:,i],lags), color = 'red', label='Unthinned Combined', lw = 3)
        ax3.legend(prop={'size':25})
        ax3.set_title(r'$\theta_{' + repr(i+1) + '}$ Autocorrelation | ' + mdl + ' ' + cStr + ' Samples (' + fStr + '% Thinning) | ID: ' + cityStr, fontsize=50)
        ax3.set_xlabel('Lag', fontsize=40)
        ax3.set_ylabel('Normalized Sample Autocorrelation', fontsize=40)
        ax3.set_xlim(left=0,right=lags)
        plt.savefig(basePath + '/' + mdl + '/' + ix + '/ACF_' + str(i) + '.png');      
        plt.clf()
        
        md.append(cHDI)
        mn.append(np.mean(dh))
        sd.append(np.std(dh,ddof=1))
        hd.append(rHDI-lHDI)
        hc.append((rHDI+lHDI)/2)
        iq.append(rIQR-lIQR)
        pm.append(i)
        
    return pd.DataFrame({'Median': md,'Mean': mn,'STD': sd,'HDC': hc,'HDI': hd,'IQR': iq,'Theta': pm})

# Data Wrangling

### Pickle and Combine Parallel MCMC Chains

In [15]:
# variable declarations
mdlList = ['ARCN','MBCN']
nChains = 4
mm = 3
kk = 4

burnVal = int(1e6) # burn-in period
warmVal = int(1e6) # number of post-burn-in samples
thinVal = int(1e5) # number target samples retained post-burn-in
ppdVal = int(1e4)
thinFrac = thinVal/warmVal; # fraction retained from warmed samples
cVal = int(thinVal*nChains) # total number of thinned samples aggregated parallel chains

accT = 0.234
lags = 100

N = int(warmVal + burnVal)

nStr = '%.0e' % N
tStr = '%.0e' % thinVal
cStr = '%.0e' % cVal
fStr = str(int(100*thinFrac))
 
cityNames = pd.read_pickle('Climo.pkl').City.unique()

varNames = ['Tmax','Tmin','Vmax']
unitNames = ['[K]','[K]','[m/s]']

betaList = ['B_0','B_1','B_2','B_3','B_4','B_5','B_6',
            'B_7','B_8','B_9','B_10','B_11','B_12',
            'B_13','B_14','B_15','B_16','B_17']
cityList = ['HAR','GRI','RNO','TVC','SEA','JAN','BIS','BNA','DFW']
ARList = ['ARN1', 'ARN2', 'ARN3', 'ARN4', 'ARN5', 'ARN6','ARP1', 'ARP2', 'ARP3', 'ARP4', 'ARP5', 'ARP6']
MBList = ['MBN1', 'MBN2', 'MBN3', 'MBN4', 'MBN5', 'MBN6','MBP1', 'MBP2', 'MBP3', 'MBP4', 'MBP5', 'MBP6']
SRList = ARList + MBList

In [16]:
for j, mdl in enumerate(mdlList):
    
    if not os.path.exists(mdl):
    
        os.makedirs(mdl)

    dfzs = pd.read_pickle(mdl + '_zscores.pkl')

    for i in range(nChains):

        dfth_temp = pd.read_pickle(mdl + '_params_' + str(i) + '.pkl')

        if i == 0:

            dfth = dfth_temp.copy()

        else:

            dfth = pd.concat((dfth,dfth_temp), ignore_index=True)

        del dfth_temp
        
    dfth['Model'] = mdl  
    dfzs['Model'] = mdl
    
    dfs = dfth[dfth['Proposal'] >= burnVal]
    
    dfs = dfs.set_index(['Model','Chain','City','Proposal'])
    dfth = dfth.set_index(['Model','Chain','City','Proposal'])
    dfzs = dfzs.set_index(['Model','Type','Stat','City'])
    
    dfs = dfs.loc[:,betaList]
    
    dfs.sort_index(inplace=True)
    dfth.sort_index(inplace=True)
    dfzs.sort_index(inplace=True)
    
    dfs.to_pickle(mdl + '_TH.pkl')
    dfzs.to_pickle(mdl + '_ZS.pkl')

    del dfzs
    del dfs

In [ ]:
    ix = 0

    for idx, dx in dfth.groupby(level=['City','Chain']):

        dt = dx.drop(['Accept','Width'], axis=1).ix[burnVal:]

        # n1 = dt.ix[::10]
        # n2 = dt.ix[::100]
        # n3 = dt.ix[::1000]
        # n4 = dt.ix[::10000]
        # n5 = dt.ix[::100000]

        n1 = dt.ix[:10]
        n2 = dt.ix[:100]
        n3 = dt.ix[:1000]
        n4 = dt.ix[:10000]
        n5 = dt.ix[:100000]

        # n1 = dt.ix[npr.choice(np.arange(0,burnVal),burnVal/10,replace=False).astype(int)]
        # n2 = dt.ix[npr.choice(np.arange(0,burnVal),burnVal/100,replace=False).astype(int)]
        # n3 = dt.ix[npr.choice(np.arange(0,burnVal),burnVal/1000,replace=False).astype(int)]
        # n4 = dt.ix[npr.choice(np.arange(0,burnVal),burnVal/10000,replace=False).astype(int)]
        # n5 = dt.ix[npr.choice(np.arange(0,burnVal),burnVal/100000,replace=False).astype(int)]

        if ix == 0:

            N1 = n1.copy()
            N2 = n2.copy()
            N3 = n3.copy()
            N4 = n4.copy()
            N5 = n5.copy()

        else:

            N1 = pd.concat((N1,n1))
            N2 = pd.concat((N2,n2))
            N3 = pd.concat((N3,n3))
            N4 = pd.concat((N4,n4))
            N5 = pd.concat((N5,n5))

        ix += 1

In [ ]:
# compute groups statistics for each level of thinning (i.e., n = 1, 2,..., 5)
Mj1 = N1.groupby(level=['City','Chain']).mean()
Sj1 = N1.groupby(level=['City','Chain']).var()
GR1 = pd.concat([Mj1,Sj1],keys=['Mean', 'Variance'],names=['Stat']).reorder_levels(['City','Stat','Chain'],axis=0)

Mj2 = N2.groupby(level=['City','Chain']).mean()
Sj2 = N2.groupby(level=['City','Chain']).var()
GR2 = pd.concat([Mj2,Sj2],keys=['Mean', 'Variance'],names=['Stat']).reorder_levels(['City','Stat','Chain'],axis=0)

Mj3 = N3.groupby(level=['City','Chain']).mean()
Sj3 = N3.groupby(level=['City','Chain']).var()
GR3 = pd.concat([Mj3,Sj3],keys=['Mean', 'Variance'],names=['Stat']).reorder_levels(['City','Stat','Chain'],axis=0)

Mj4 = N4.groupby(level=['City','Chain']).mean()
Sj4 = N4.groupby(level=['City','Chain']).var()
GR4 = pd.concat([Mj4,Sj4],keys=['Mean', 'Variance'],names=['Stat']).reorder_levels(['City','Stat','Chain'],axis=0)

Mj5 = N5.groupby(level=['City','Chain']).mean()
Sj5 = N5.groupby(level=['City','Chain']).var()
GR5 = pd.concat([Mj5,Sj5],keys=['Mean', 'Variance'],names=['Stat']).reorder_levels(['City','Stat','Chain'],axis=0)

GRA = pd.concat([GR1,GR2,GR3,GR4,GR5],keys=[1,2,3,4,5],names=['Order']).reorder_levels(['Order','City','Stat','Chain'],axis=0)

# mean within-chain variance
W = GRA.xs('Variance',level='Stat').groupby(level=['Order','City']).mean()

# between-chain mean of within-chain means
ensMean = GRA.xs('Mean',level='Stat').groupby(level=['Order','City']).mean()

# between-chain variance of within-chain means
B = burnVal*GRA.xs('Mean',level='Stat').groupby(level=['Order','City']).var()

# potential scale reduction factor
PSRF = ((burnVal - 1)/burnVal + (nChains + 1)*B/(W*nChains*burnVal))**0.5

PSRF.columns = list(np.arange(0,18))
df = PSRF.reset_index()
cols = list(df)
cols = cols[2:]
dm = pd.melt(df,id_vars=['Order','City'], value_vars=cols)

In [ ]:
ts = 70
fs = 70
ls = 35

for idx, dx in dm.groupby('City'):
    
    plt.figure(figsize=(32,24))
    pvt = dx.pivot("variable", "Order", "value")
    sns.heatmap(pvt, annot=True, fmt='.1f')
    plt.title('Gelman–Rubin Convergence | PSRF | ID: ' + mdl + ' ' + idx, fontsize=ts)
    plt.xlabel('Thinning Order: $10^n$ Samples', fontsize=fs)
    plt.ylabel(r'Model Parameters: $\beta_i$',fontsize=fs)
    plt.xticks(fontsize = ls)
    plt.yticks(fontsize = ls)
    plt.gca().invert_yaxis()

In [ ]:
ia = np.arange(1,1e2)
ib = np.arange(1e2,1e3,10)
ic = np.arange(1e3,N+1,100)
iv = np.concatenate((ia,ib,ic), axis=0)

ii = 0

for ix, dx in dfth.groupby(level=['City']):
    
    cityStr = ix
    
    if not os.path.exists(mdl + '/' + ix):
        
        os.makedirs(mdl + '/' + ix)
    
    dx = dx.reset_index()
    dt = dx.drop(['Accept','Width'], axis=1)
    dth = dt[dt['Proposal']>=burnVal].reset_index(drop=True)
    idx = npr.choice(np.arange(dth.shape[0]),thinVal*nChains,replace=False).astype(int)
    th = dth.iloc[idx,3:]
    xh = dth.iloc[:,3:]

    # Convergence Diagnostics
    plot_diag(dx,burnVal,N,ix,basePath)
    
    # PCC Plots
    pccAll(th,3,ix,basePath)
    
    # Individual Parameter Plots
    tempDF = plot_theta(th,dt,dth,burnVal,N,lags,idx,ix,basePath)
    tempDF['City'] = ix
    
    if ii == 0:
        
        dfPost = tempDF.copy()
        
    else:
        
        dfPost = pd.concat((dfPost,tempDF), ignore_index=True)
        
    ii += 1

In [ ]:
ts = 65
fs = 65
ls = 30

plt.figure(figsize=(32,24))
pvt = dfPost.pivot("Theta", "City", "Median")
sns.heatmap(pvt, annot=True, fmt='.1f', center=0)
plt.title('Posterior Belief | ' + mdl + ' Parameters | Median', fontsize=ts)
plt.xlabel('City', fontsize=50)
plt.ylabel(r'Model Parameters: $\beta_i$',fontsize=fs)
plt.xticks(fontsize = ls)
plt.yticks(fontsize = ls)
plt.gca().invert_yaxis()

plt.figure(figsize=(32,24))
pvt = dfPost.pivot("Theta", "City", "Mean")
sns.heatmap(pvt, annot=True, fmt='.1f', center=0)
plt.title('Posterior Belief | ' + mdl + ' Parameters | Mean', fontsize=ts)
plt.xlabel('City', fontsize=50)
plt.ylabel(r'Model Parameters: $\beta_i$',fontsize=fs)
plt.xticks(fontsize = ls)
plt.yticks(fontsize = ls)
plt.gca().invert_yaxis()

plt.figure(figsize=(32,24))
pvt = dfPost.pivot("Theta", "City", "HDC")
sns.heatmap(pvt, annot=True, fmt='.1f', center=0)
plt.title(r'Posterior Belief | ' + mdl + ' Parameters | $HDI_{95\%}$ Center', fontsize=ts)
plt.xlabel('City', fontsize=50)
plt.ylabel(r'Model Parameters: $\beta_i$',fontsize=fs)
plt.xticks(fontsize = ls)
plt.yticks(fontsize = ls)
plt.gca().invert_yaxis()

plt.figure(figsize=(32,24))
pvt = dfPost.pivot("Theta", "City", "STD")
sns.heatmap(pvt, annot=True, fmt='.1f', center=0)
plt.title(r'Posterior Belief | ' + mdl + ' Parameters | $\sigma$', fontsize=ts)
plt.xlabel('City', fontsize=50)
plt.ylabel(r'Model Parameters: $\beta_i$',fontsize=fs)
plt.xticks(fontsize = ls)
plt.yticks(fontsize = ls)
plt.gca().invert_yaxis()

plt.figure(figsize=(32,24))
pvt = dfPost.pivot("Theta", "City", "HDI")
sns.heatmap(pvt, annot=True, fmt='.1f', center=0)
plt.title(r'Posterior Belief | ' + mdl + ' Parameters | $HDI_{95\%}$', fontsize=ts)
plt.xlabel('City', fontsize=50)
plt.ylabel(r'Model Parameters: $\beta_i$',fontsize=fs)
plt.xticks(fontsize = ls)
plt.yticks(fontsize = ls)
plt.gca().invert_yaxis()

plt.figure(figsize=(32,24))
pvt = dfPost.pivot("Theta", "City", "IQR")
sns.heatmap(pvt, annot=True, fmt='.1f', center=0)
plt.title('Posterior Belief | ' + mdl + ' Parameters | IQR', fontsize=ts)
plt.xlabel('City', fontsize=50)
plt.ylabel(r'Model Parameters: $\beta_i$',fontsize=fs)
plt.xticks(fontsize = ls)
plt.yticks(fontsize = ls)
plt.gca().invert_yaxis()